In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.DataFrame({"Id": [0,1,2,3], "Val1": [1,2,3,1], "Val2": [2,3,5,3]})
df2 = pd.DataFrame({"Id": [0,1,2,3], "Val3": [3,5,4,5]})

In [ ]:
df = pd.merge(left=df1, right=df2, how='inner')

df

,Id,Val1,Val2,Val3
0,0,1,2,3
1,1,2,3,5
2,2,3,5,4
3,3,1,3,5


In [ ]:
from sklearn import linear_model
data = pd.DataFrame(columns = missing_columns)


In [ ]:
train = pd.read_csv('train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68613 entries, 0 to 68612
Data columns (total 14 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   MOFname                                        68613 non-null  object 
 1   volume [A^3]                                   68613 non-null  float64
 2   weight [u]                                     68613 non-null  float64
 3   surface_area [m^2/g]                           68613 non-null  float64
 4   void_fraction                                  68613 non-null  float64
 5   void_volume [cm^3/g]                           68613 non-null  float64
 6   functional_groups                              68290 non-null  object 
 7   metal_linker                                   68613 non-null  int64  
 8   organic_linker1                                68613 non-null  int64  
 9   organic_linker2                                686

In [ ]:
train.describe()

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
count,68613.000000,68613.000000,68613.000000,68613.000000,68613.000000,68613.000000,68613.000000,68613.000000,68613.000000,6.652600e+04,68613.000000
mean,3447.363207,1656.761858,1666.766690,0.259164,0.376052,4.203271,11.919257,20.583592,28.599681,inf,120.002797
std,4840.665782,1259.086320,1366.317223,0.164758,0.476452,3.144905,10.783136,10.100870,153.806887,NaN,89.573112
min,606.576038,439.281220,-1.000000,-1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.612299e+00,-44.285746
25%,1556.075767,893.662700,511.930000,0.142580,0.119000,2.000000,4.000000,14.000000,12.818366,5.267536e+00,65.537205
50%,2190.442847,1259.699253,1542.830000,0.242620,0.248400,3.000000,10.000000,20.000000,19.689890,5.899089e+00,98.552185
75%,3605.836441,1897.163660,2517.960000,0.345120,0.443800,4.000000,16.000000,25.000000,32.954388,6.768365e+00,163.139540
max,223964.854408,22595.928960,7083.530000,0.872060,6.610100,12.000000,59.000000,59.000000,29369.777780,inf,736.061636


## Imputation

In [ ]:
train.loc[train["surface_area [m^2/g]"] == 0, "surface_area [m^2/g]"] = np.NAN
train.loc[train["surface_area [m^2/g]"] == -1, "surface_area [m^2/g]"] = np.NAN
train.loc[train["void_fraction"] == 0, "void_fraction"] = np.NAN
train.loc[train["void_fraction"] == -1, "void_fraction"] = np.NAN
train.loc[train["void_volume [cm^3/g]"] == 0, "void_volume [cm^3/g]"] = np.NAN
train.loc[train["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] == np.inf, "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] = np.nan


In [ ]:
for col in ['metal_linker', 'organic_linker1', 'organic_linker2']:
    train[col] = train[col].astype('category')

In [ ]:
train.isnull().sum()

MOFname                                              0
volume [A^3]                                         0
weight [u]                                           0
surface_area [m^2/g]                             14539
void_fraction                                     2930
void_volume [cm^3/g]                              3130
functional_groups                                  323
metal_linker                                         0
organic_linker1                                      0
organic_linker2                                      0
topology                                             0
CO2/N2_selectivity                                   0
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]     2089
CO2_working_capacity [mL/g]                          0
dtype: int64

In [ ]:
missing_columns = ["surface_area [m^2/g]", "void_fraction", "void_volume [cm^3/g]"]
cate_columns = ['topology','functional_groups','MOFname','metal_linker', 'organic_linker1', 'organic_linker2']

In [ ]:
train = train.dropna(subset=['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'])

In [ ]:
prepare_train = train.copy()

In [ ]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

In [ ]:
for feature in missing_columns:
    train[feature + '_imp'] = train[feature]
    train = random_imputation(train, feature)

## Trainning set

In [ ]:

for feature in missing_columns:
        
    data[feature] = train[feature+'_imp']
    parameters = list(set(train.columns) - set(missing_columns)-set(cate_columns)-{feature + '_imp'} - set(['CO2_working_capacity [mL/g]']))
    #print(train[parameters])
    #Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(train[parameters],train[feature + '_imp'])
    
    #observe that I preserve the index of the missing data from the original dataframe
    data.loc[train[feature].isnull(),feature] = model.predict(train[parameters])[train[feature].isnull()]

In [ ]:
for column in missing_columns:
    prepare_train[column] = data[column]

In [ ]:
prepare_train.dropna(inplace=True)

In [ ]:
prepare_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66203 entries, 0 to 66523
Data columns (total 14 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   MOFname                                        66203 non-null  object  
 1   volume [A^3]                                   66203 non-null  float64 
 2   weight [u]                                     66203 non-null  float64 
 3   surface_area [m^2/g]                           66203 non-null  float64 
 4   void_fraction                                  66203 non-null  float64 
 5   void_volume [cm^3/g]                           66203 non-null  float64 
 6   functional_groups                              66203 non-null  object  
 7   metal_linker                                   66203 non-null  category
 8   organic_linker1                                66203 non-null  category
 9   organic_linker2                        

In [ ]:
prepare_train.to_csv('prepare_train.csv',index=False)

# Test-set

In [ ]:
test = pd.read_csv('pretest.csv')

In [ ]:
test.loc[test["surface_area [m^2/g]"] == 0, "surface_area [m^2/g]"] = np.NAN
test.loc[test["surface_area [m^2/g]"] == -1, "surface_area [m^2/g]"] = np.NAN
test.loc[test["void_fraction"] == 0, "void_fraction"] = np.NAN
test.loc[test["void_fraction"] == -1, "void_fraction"] = np.NAN
test.loc[test["void_volume [cm^3/g]"] == 0, "void_volume [cm^3/g]"] = np.NAN
test.loc[test["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] == np.inf, "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] = np.nan


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   MOFname                                        2000 non-null   object 
 1   volume [A^3]                                   2000 non-null   float64
 2   weight [u]                                     2000 non-null   float64
 3   surface_area [m^2/g]                           1688 non-null   float64
 4   void_fraction                                  1973 non-null   float64
 5   void_volume [cm^3/g]                           1972 non-null   float64
 6   functional_groups                              1979 non-null   object 
 7   metal_linker                                   2000 non-null   int64  
 8   organic_linker1                                2000 non-null   int64  
 9   organic_linker2                                2000 

In [ ]:
prepare_test = test.copy()

In [ ]:
for feature in missing_columns:
    test[feature + '_imp'] = test[feature]
    test = random_imputation(test, feature)

## Deterministic test set

In [ ]:
data_test = pd.DataFrame(columns = missing_columns)

for feature in missing_columns:
        
    data_test[feature] = test[feature+'_imp']
    parameters = list(set(test.columns) - set(missing_columns)-set(cate_columns)-{feature + '_imp'} - set('CO2_working_capacity [mL/g]'))
    
    #observe that I preserve the index of the missing data_test from the original data_testframe
    data_test.loc[test[feature].isnull(),feature] = model.predict(test[parameters])[test[feature].isnull()]

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/base.py:484: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- void_volume [cm^3/g]_imp
Feature names seen at fit time, yet now missing:
- surface_area [m^2/g]_imp

  warnings.warn(message, FutureWarning)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/base.py:484: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- void_volume [cm^3/g]_imp
Feature names seen at fit time, yet now missing:
- void_fraction_imp

  warnings.warn(message, FutureWarning)


In [ ]:
for column in missing_columns:
    prepare_test[column] = data_test[column]

In [ ]:
prepare_test.to_csv('prepare_test.csv',index=False)

## stochastic impute

In [ ]:
random_data = pd.DataFrame(columns = [name for name in missing_columns])

for feature in missing_columns:
        
    random_data[feature] = train[feature + '_imp']
    parameters = list(set(train.columns) - set(missing_columns)-set(cate_columns)-{feature + '_imp'} - set(['CO2_working_capacity [mL/g]']))
    
    model = linear_model.LinearRegression()
    model.fit(X = train[parameters], y = train[feature + '_imp'])
    
    #Standard Error of the regression estimates is equal to std() of the errors of each estimates
    predict = model.predict(train[parameters])
    std_error = (predict[train[feature].notnull()] - train.loc[train[feature].notnull(), feature + '_imp']).std()
    
    #observe that I preserve the index of the missing data from the original dataframe
    random_predict = np.random.normal(size = train[feature].shape[0], 
                                      loc = predict, 
                                      scale = std_error)
    random_data.loc[(train[feature].isnull()) & (random_predict > 0), feature] = random_predict[(train[feature].isnull()) & 
                                                                            (random_predict > 0)]

In [ ]:
for column in missing_columns:
    prepare_train[column] = random_data[column]

In [ ]:
prepare_train.dropna(inplace=True)
prepare_train.to_csv('train_impute_stochastic_wo_smiles.csv',index=False)

### test set

In [ ]:
data_test = pd.DataFrame(columns = missing_columns)

for feature in missing_columns:
        
    data_test[feature] = test[feature+'_imp']
    parameters = list(set(test.columns) - set(missing_columns)-set(cate_columns)-{feature + '_imp'} - set('CO2_working_capacity [mL/g]'))
    
    #observe that I preserve the index of the missing data_test from the original data_testframe
    data_test.loc[test[feature].isnull(),feature] = model.predict(test[parameters])[test[feature].isnull()]
    random_data.loc[(train[feature].isnull()) & (random_predict > 0), feature] = random_predict[(train[feature].isnull()) & 
                                                                            (random_predict > 0)]

In [ ]:
for column in missing_columns:
    prepare_test[column] = data_test[column]
prepare_test.to_csv('pretest_impute_stochastic_wo_smile.csv',index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa57721c-b209-430d-97e4-796ac27ee623' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>